In [1]:
import os
import time
import contextlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras, feature_column
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import PIL.Image as Image
import tensorflow_datasets as tfds

from IPython.core.interactiveshell import InteractiveShell

In [2]:
# 配置项
# 这个要放到设置中文之前否则还是小方框
plt.style.use("seaborn")

# 指定默认字体 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
InteractiveShell.ast_node_interactivity = "all"

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 1s 1us/step


In [4]:
path_to_file

'/Users/JQC/.keras/datasets/shakespeare.txt'

In [7]:
df = pd.read_table(path_to_file)
df.shape
df.head()

(32776, 1)

,First Citizen:
0,"Before we proceed any further, hear me speak."
1,All:
2,"Speak, speak."
3,First Citizen:
4,You are all resolved rather to die than to fam...


In [9]:
text = open(path_to_file, 'rb').read().decode('utf-8')
print('字符长度:{}'.format(len(text)))

字符长度:1115394


In [11]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [12]:
# 无重复的词汇
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


65 unique characters


In [36]:
# 将字符转化为对应的下标
char2idx = {word: i for i, word in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[char] for char in text])


In [29]:
for char, _ in zip(char2idx, range(10)):
    print('{:4s}: {:d}'.format(repr(char), char2idx[char]))

'\n': 0
' ' : 1
'!' : 2
'$' : 3
'&' : 4
"'" : 5
',' : 6
'-' : 7
'.' : 8
'3' : 9


In [37]:
seq_length = 100
examples_per_epoch = len(text) // seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i


In [39]:
# 101个字符画一个批次
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)
for item in sequences.take(1):
    temp = repr(''.join(idx2char[item.numpy()]))       
    print(len(idx2char[item.numpy()]))
    print(temp)        

101
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [40]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [43]:
# RNN除了要考虑当前还是考虑上一个
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [42]:
# 随机打乱数据 在划分批次
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [44]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.LSTM(rnn_units, 
                          return_sequences=True, 
                          stateful=True, 
                          recurrent_initializer='glorot_uniform'),
        keras.layers.Dense(vocab_size)
    ])
    return model

In [47]:
model = build_model(vocab_size=vocab_size,
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units, 
                    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [49]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# 删除为为1的维度
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
len(sampled_indices)
sampled_indices

100

array([59, 20, 26, 38,  1, 17, 37, 20, 14, 20,  0, 18, 37, 12, 33, 30,  4,
        8, 26,  0, 47, 58, 43, 18, 26, 37, 54, 53, 24, 20, 24, 29, 18, 62,
       53, 16, 22, 54,  8, 27, 37, 63, 46, 33, 43, 51, 49, 61,  6, 49, 17,
       29, 11, 59, 54, 30, 33, 60, 62, 53, 61, 19,  0,  3, 15, 34,  8, 61,
       16, 48, 54, 57, 30, 50, 60, 38, 47, 48,  0, 37,  3, 29, 64, 53,  4,
       56, 33, 27, 48,  7, 47,  5, 41, 35, 47, 43, 61, 20, 31, 19])

In [53]:
# 解码未训练模型的预测值
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'e misfortune live?\n\nGroom:\nI was a poor groom of thy stable, king,\nWhen thou wert king; who, travell'

Next Char Predictions: 
 "-3:;duk\nz3e-vr3xtBfHglyLggibLHniYBPx:yjp$'IwgeOv$tGpTzovVkiE!CWJLYG!eVHq$g'ccYhd zwVcSaqEsUmgSC:!AXz"


In [59]:
# 训练模型 损失函数 优化器
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1732244


In [61]:
# 编译模型
model.compile(optimizer='adam', loss=loss)

# 训练过程中保存checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_prefix, save_weights_only=True)
EPOCHS=10

# 训练模型
model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 693s 4s/step - loss: 2.5483
Epoch 2/10
172/172 [==============================] - 692s 4s/step - loss: 1.8608
Epoch 3/10
172/172 [==============================] - 2356s 14s/step - loss: 1.6181
Epoch 4/10
 74/172 [===========>..................] - ETA: 7:29 - loss: 1.5108

KeyboardInterrupt: 

In [ ]:
# 根据前面训练的模型生成文本
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # 将初始化的字符转化为数字类型的
  input_eval = [char2idx[s] for s in start_string]
  # 后面补0升维
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      # 删除第0维为1的
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
# 更高级的方法:自定义训练 使用梯度下降最小化损失函数来逼近数据
# 建立模型
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

# 编译
optimizer = tf.keras.optimizers.Adam()

# 自定义损失函数
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()
  # 数据批次训练
  for (batch_n, (inp, target)) in enumerate(dataset):
    # 计算loss
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))